<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/analyze_rheobase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
########## Rheobase analysis ################
'requires: numpy, pandas, matplotlib, ...'

def analyze_rheo(file_name,
                 spike_args =  {'spike_thresh':10, 'high_dv_thresh': 25,'low_dv_thresh': -5,'window_ms': 2},
                 to_plot=False,
                 verbose=False,
                 single_spike=True):
    'Analyze single abf of current injections for rheobase'
    'single_spike: look for one and only one spike, no doublets'


    abf = abf_or_name(file_name)
    if to_plot: plot_sweeps_and_command(abf)
    if len(abf.sweepList)<2:
        return np.nan,np.nan,np.nan,np.nan,np.nan  
    else:
        is_base, is_stim = protocol_baseline_and_stim(abf)
        stim_currents, spike_counts, spike_rates, V_before,_,_ = spikes_per_stim(abf, spike_args,thresh=20,to_plot=to_plot)
        if verbose: print(spike_counts)
        single_spikes = spike_counts==1
        zero_spikes = spike_counts==0
        if single_spike:
            none_to_one = np.full(single_spikes.shape, False)
            none_to_one[1:] = np.logical_and(single_spikes[1:], zero_spikes[:-1])
            first_spike_stim = np.where(none_to_one)[0]
        else:
            some_spikes = spike_counts>0
            none_to_some = np.full(single_spikes.shape, False)
            none_to_some[1:] = np.logical_and(some_spikes[1:], zero_spikes[:-1])
            first_spike_stim = np.where(none_to_some)[0]
    if first_spike_stim.size == 0:
        return np.nan,np.nan,np.nan,np.nan,np.nan 
    else:
        if first_spike_stim.size >1:
            first_spike_stim = np.min(first_spike_stim)
        rheo = stim_currents[first_spike_stim]
        _, _, _, QC_val_df = Iclamp_QC(file_name)
        offset = np.mean(QC_val_df['I_leak'])
        Vhold_spike = QC_val_df['V_hold'][first_spike_stim]
        ap_thresh = V_before[first_spike_stim]
        step_resolution = np.mean(np.diff(stim_currents))

    return rheo, step_resolution, offset, Vhold_spike, ap_thresh 


